In [1]:
import re
import pandas as pd

In [2]:
f = open('chat.txt','r', encoding='utf-8')

In [3]:
data = f.read()

In [4]:
data

'User: Hi, can you tell me about Python?,\nAI: Sure! Python is a popular programming language known for its readability,\nUser: What can I use it for?,\nAI: You can use Python for web development, data analysis, AI, and more,'

In [5]:
lines = data.split('\n')
df = pd.DataFrame({'user_message': lines})

In [6]:
df.head()

,user_message
0,"User: Hi, can you tell me about Python?,"
1,AI: Sure! Python is a popular programming lang...
2,"User: What can I use it for?,"
3,"AI: You can use Python for web development, da..."


In [7]:
df.shape

(4, 1)

In [8]:
# Split user and message
users = []
messages = []

for message in df['user_message']:
    match = re.match(r'^(\w+):\s(.+)', message.strip())
    if match:
        users.append(match.group(1))       
        messages.append(match.group(2))    
df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()

,user,message
0,User,"Hi, can you tell me about Python?,"
1,AI,Sure! Python is a popular programming language...
2,User,"What can I use it for?,"
3,AI,"You can use Python for web development, data a..."


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

documents = df['message'].tolist()

# Load NLTK English stopwords
default_stopwords = set(stopwords.words('english'))

# Add custom generic/common words
custom_words = {
    'hi', 'popular', 'known', 'can', 'use', 'tell', 'sure', 'also', 'get', 'like',
    'one', 'many', 'used', 'based', 'etc', 'really', 'know', 'would'
}

final_stopwords = list(default_stopwords.union(custom_words))

# Initialize TF-IDF vectorizer with custom stopwords list
vectorizer = TfidfVectorizer(stop_words=final_stopwords)
X = vectorizer.fit_transform(documents)

# Extract keywords and scores
feature_names = vectorizer.get_feature_names_out()
scores = X.toarray().sum(axis=0)

tfidf_scores = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
top_keywords = [word for word, _ in tfidf_scores[:5]]

print("Top keywords after removing stopwords and generic terms:")
for word, score in tfidf_scores[:5]:
    print(f"{word}: {score:.4f}")

print(top_keywords)

Top keywords after removing stopwords and generic terms:
python: 1.6203
language: 0.5417
programming: 0.5417
readability: 0.5417
ai: 0.4300
['python', 'language', 'programming', 'readability', 'ai']


In [11]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 108.9 kB/s eta 0:01:58
     --------------------------------------- 0.0/12.8 MB 164.3 kB/s eta 0:01:18
     --------------------------------------- 0.1/12.8 MB 204.8 kB/s eta 0:01:03
     --------------------------------------- 0.1/12.8 MB 275.8 kB/s eta 0:00:47
     --------------------------------------- 0.1/12.8 MB 387.0 kB/s eta 0:00:33
     --------------------------------------- 0.2/12.8 MB 382.3 kB/s eta 0:00:34
      -------------------------------------- 0.2/12.8 MB 497.3 kB/s eta 0:00:26
      -------------------------------------- 0.3/12.8 MB 575.5 kB/s eta 0:00:22
     - ------------------------------------- 0.4/12.8 MB 655.2 kB/s eta

In [22]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Combine all messages into a single text block
full_text = " ".join(df['message'].tolist())

# Process text with spaCy
doc = nlp(full_text)

# Generate keyword list
keywords = Counter(noun_chunks).most_common(5)
keyword_list = [kw[0] for kw in keywords]

# Extract named entities and noun chunks to infer topic
entities = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PERSON", "PRODUCT", "LANGUAGE", "WORK_OF_ART"]]
noun_chunks = [chunk.text.lower() for chunk in doc.noun_chunks if len(chunk.text.split()) > 1]

summary = f"""
The user asked mainly about {', '.join(set(entities + keyword_list))}.
"""
print(summary)


The user asked mainly about data analysis, Python, its readability, web development, a popular programming language.

